In [97]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
import scipy.signal as ssignal
from scipy.fft import fft, fftfreq
# from scipy.signal import resample, butter, filtfilt, medfilt 
# from scipy.ndimage import gaussian_filter1d

In [98]:
# def funcBPFilter(ppg_signal, order, f1, f2, fs):
#     fc = [f1, f2]
#     b, a = butter(order, fc, btype='bandpass', analog=False, output='ba', fs=fs)
#     signalBP = filtfilt(b, a, ppg_signal)
#     return signalBP

# def funcLowPassFilter(ppg_signal, order, cutoff, fs):
#     b, a = butter(order, cutoff, btype='lowpass', analog=False, output='ba', fs=fs)
#     signalLP = filtfilt(b, a, ppg_signal)
#     return signalLP

# def funcHighPassFilter(ppg_signal, order, cutoff, fs):
#     b, a = butter(order, cutoff, btype='highpass', analog=False, output='ba', fs=fs)
#     signalHP = filtfilt(b, a, ppg_signal)
#     return signalHP

# def funcMedianFilter(ppg_signal, window_size):
#     return medfilt(ppg_signal, kernel_size=window_size)

# def funcExponentialFilter(ppg_signal, alpha):
#     filtered_signal = np.zeros_like(ppg_signal)
#     filtered_signal[0] = ppg_signal[0]
#     for i in range(1, len(ppg_signal)):
#         filtered_signal[i] = alpha * ppg_signal[i] + (1 - alpha) * filtered_signal[i-1]
#     return filtered_signal

def cheby(ppg, fs, order=4, rs=20, btype='low'):
    nyquist = 0.5 * fs
    cutoff = 10 / nyquist
    rs, order = 20, 4
    b, a = ssignal.cheby2(N=order, rs=rs, Wn=cutoff, btype='low')
    return ssignal.filtfilt(b, a, ppg)

def butter(data, fs, order=4, lowcut=0.5, highcut=3):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = ssignal.butter(order, [low, high], btype='bandpass')
    return ssignal.filtfilt(b, a, data)

In [99]:
age_group = {15:1.0,16:1.0,17:1.0,18:1.0,19:1.0,
             20:2.0,21:2.0,22:2.0,23:2.0,24:2.0,
             25:3.0,26:3.0,27:3.0,28:3.0,29:3.0,
             30:4.0,31:4.0,32:4.0,33:4.0,34:4.0,
             35:5.0,36:5.0,37:5.0,38:5.0,39:5.0,
             40:6.0,41:6.0,42:6.0,43:6.0,44:6.0,
             45:7.0,46:7.0,47:7.0,48:7.0,49:7.0,
             50:8.0,51:8.0,52:8.0,53:8.0,54:8.0,
             55:9.0,56:9.0,57:9.0,58:9.0,59:9.0,
             60:10.0,61:10.0,62:10.0,63:10.0,64:10.0,
             65:11.0,66:11.0,67:11.0,68:11.0,69:11.0,
             70:12.0,71:12.0,72:12.0,73:12.0,74:12.0,
             75:13.0,76:13.0,77:13.0,78:13.0,79:13.0,
             80:14.0,81:14.0,82:14.0,83:14.0,84:14.0,
             85:15.0,86:15.0,87:15.0,88:15.0,89:15.0,
             90:16.0,91:16.0,92:16.0,93:16.0,94:16.0,
             95:17.0,96:17.0,97:17.0,98:17.0,99:17.0,100:17.0,
             }

In [ ]:
subjects = pd.read_csv(r"D:/CK/bruno/subject-info.csv",dtype = {"ID":str})
subjects.dropna(subset=["Age [years]"],inplace=True)
subjects = subjects.reset_index()
subjects = subjects[["ID","Age [years]"]]
subjects["Age [years]"] = subjects["Age [years]"].map(age_group)
subjects["Age [years]"]

0       8.0
1       8.0
2       8.0
3       8.0
4       8.0
       ... 
3883    8.0
3884    8.0
3885    8.0
3886    8.0
3887    8.0
Name: Age [years], Length: 3888, dtype: float64

In [101]:
ids = subjects["ID"].values
age_groups = subjects["Age [years]"].values

In [ ]:
result = []
result_metadata = []

new_id = 1147

for id in ids:
    new_id += 1 
    try:
        record = wfdb.rdrecord('D:/CK/bruno/' + id + "/"+ id+"_PPG")

        #print(record.__dict__)
        #print(record.p_signal.shape)
        #channel_index = record.sig_name.index('NIBP')
        signal = record.p_signal.astype(np.float32).flatten()
        fs = record.fs

        five_minute_samples = 60 * 5 * fs

        signal = signal[:five_minute_samples]

        # Применение различных фильтров
        #signal = funcBPFilter(signal, order=2, f1=0.5, f2=4.0, fs=fs)
        # signal_lp = funcLowPassFilter(signal, order=2, cutoff=4.0, fs=fs)
        # signal_hp = funcHighPassFilter(signal, order=2, cutoff=0.5, fs=fs)
        #signal = funcMedianFilter(signal, window_size=5)
        # signal_exponential = funcExponentialFilter(signal, alpha=0.1)

        # Выбор фильтра для дальнейшей обработки
        signal = cheby(signal,fs,order = 4)
        #signal = butter(signal,fs,3)

        try:
            wd, m = hp.process(signal, fs)
            
            # Вычисляем частотные характеристики с помощью scipy
            n = len(signal)
            yf = fft(signal)
            xf = fftfreq(n, 1/fs)
            
            # Определяем границы для LF и HF
            lf_band = (0.04, 0.15)
            hf_band = (0.15, 0.4)
            
            # Вычисляем LF и HF
            lf_mask = (xf >= lf_band[0]) & (xf <= lf_band[1])
            hf_mask = (xf >= hf_band[0]) & (xf <= hf_band[1])
            
            lf = np.trapz(np.abs(yf[lf_mask])**2, xf[lf_mask])
            hf = np.trapz(np.abs(yf[hf_mask])**2, xf[hf_mask])
            lf_hf_ratio = lf / hf
            
            freq_features = {
                'lf': lf,
                'hf': hf,
                'lf_hf_ratio': lf_hf_ratio
            }

            idx = int(subjects[subjects["ID"] == id].index.to_list()[0])
            age_group = age_groups[idx]

            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                'lf': freq_features['lf'],
                'hf': freq_features['hf'],
                'lf_hf_ratio': freq_features['lf_hf_ratio'],
                **m
            })

            print((new_id, id, age_group, freq_features['lf'], freq_features['hf'], freq_features['lf_hf_ratio']))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")

(1148, '100001', 8.0, 0.0, 9.745234558957998, 0.0)
Process end: 100001
(1149, '100002', 8.0, 0.0, 4.342184831523333, 0.0)
Process end: 100002
(1150, '100003', 8.0, 0.0, 19.86628969308419, 0.0)
Process end: 100003
(1151, '100004', 8.0, 0.0, 342007.56345063256, 0.0)
Process end: 100004
(1152, '101001', 8.0, 0.0, 1.7881711408072056, 0.0)
Process end: 101001
(1153, '101002', 8.0, 0.0, 1.2276648903973735, 0.0)
Process end: 101002
(1154, '101003', 8.0, 0.0, 14.499257645168464, 0.0)
Process end: 101003
(1155, '101004', 8.0, 0.0, 82699.8106079998, 0.0)
Process end: 101004
(1156, '102001', 10.0, 0.0, 9.204607174469253, 0.0)
Process end: 102001
(1157, '102002', 10.0, 0.0, 5.23686852458832, 0.0)
Process end: 102002


c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\heartpy\analysis.py:677: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  interp = UnivariateSpline(x, rrlist, k=3)
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:5317: RuntimeWarning: Mean of empty slice.
  result = super().m

(1158, '102003', 10.0, 0.0, 2.081093929740705, 0.0)
Process end: 102003
(1159, '102004', 10.0, 0.0, 705294.004554088, 0.0)
Process end: 102004
(1160, '103001', 2.0, 0.0, 91.80478264312971, 0.0)
Process end: 103001
(1161, '103002', 2.0, 0.0, 11.638493287670475, 0.0)
Process end: 103002
(1162, '103003', 2.0, 0.0, 8.011031821110636, 0.0)
Process end: 103003
(1163, '103004', 2.0, 0.0, 106393.3905133736, 0.0)
Process end: 103004
(1164, '104001', 2.0, 0.0, 65.83188087915754, 0.0)
Process end: 104001
(1165, '104002', 2.0, 0.0, 76.43605336123753, 0.0)
Process end: 104002
(1166, '104003', 2.0, 0.0, 398.9259232146462, 0.0)
Process end: 104003
(1167, '104004', 2.0, 0.0, 235141.6212365744, 0.0)
Process end: 104004
(1168, '105001', 2.0, 0.0, 34.166264511830796, 0.0)
Process end: 105001
(1169, '105002', 2.0, 0.0, 54.35813519542102, 0.0)
Process end: 105002
(1170, '105003', 2.0, 0.0, 22.202863495013673, 0.0)
Process end: 105003
(1171, '105004', 2.0, 0.0, 181784.14353923866, 0.0)
Process end: 105004
(

In [103]:
df_metadata = pd.DataFrame(result_metadata)
df1 = df_metadata.drop(columns=['error'])
df2 = df1.dropna()
df_metadata

,id,age_group,lf,hf,lf_hf_ratio,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,error
0,1148,8.0,0.0,9.745235,0.0,211.764706,283.333333,163.724023,146.89939,253.062056,0.909091,0.818182,116.666667,174.735797,130.443898,71607.007952,1.339547,0.441794,NaN
1,1149,8.0,0.0,4.342185,0.0,100.699301,595.833333,147.608247,84.327404,157.762128,1.000000,1.000000,83.333333,104.986772,219.089023,72261.183922,0.479197,0.209864,NaN
2,1150,8.0,0.0,19.866290,0.0,161.194030,372.222222,197.593547,109.420241,391.780632,1.000000,1.000000,200.000000,274.543530,58.708705,50636.488945,4.676368,0.448129,NaN
3,1151,8.0,0.0,342007.563451,0.0,163.636364,366.666667,44.095855,35.572912,45.133547,0.500000,0.166667,0.000000,29.658551,31.671540,2950.998393,0.936442,0.341413,NaN
4,1152,8.0,0.0,1.788171,0.0,66.176471,906.666667,35.901099,38.490018,58.794474,0.666667,0.444444,0.000000,41.573971,30.987408,4047.218364,1.341641,0.331016,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,5031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning
3884,5032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning
3885,5033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning
3886,5034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning


In [104]:
df1

,id,age_group,lf,hf,lf_hf_ratio,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,1148,8.0,0.0,9.745235,0.0,211.764706,283.333333,163.724023,146.89939,253.062056,0.909091,0.818182,116.666667,174.735797,130.443898,71607.007952,1.339547,0.441794
1,1149,8.0,0.0,4.342185,0.0,100.699301,595.833333,147.608247,84.327404,157.762128,1.000000,1.000000,83.333333,104.986772,219.089023,72261.183922,0.479197,0.209864
2,1150,8.0,0.0,19.866290,0.0,161.194030,372.222222,197.593547,109.420241,391.780632,1.000000,1.000000,200.000000,274.543530,58.708705,50636.488945,4.676368,0.448129
3,1151,8.0,0.0,342007.563451,0.0,163.636364,366.666667,44.095855,35.572912,45.133547,0.500000,0.166667,0.000000,29.658551,31.671540,2950.998393,0.936442,0.341413
4,1152,8.0,0.0,1.788171,0.0,66.176471,906.666667,35.901099,38.490018,58.794474,0.666667,0.444444,0.000000,41.573971,30.987408,4047.218364,1.341641,0.331016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,5031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3884,5032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3885,5033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3886,5034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df2

,id,age_group,lf,hf,lf_hf_ratio,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,1148,8.0,0.000000e+00,9.745235e+00,0.000000,211.764706,283.333333,163.724023,146.89939,253.062056,0.909091,0.818182,116.666667,174.735797,130.443898,71607.007952,1.339547,0.441794
1,1149,8.0,0.000000e+00,4.342185e+00,0.000000,100.699301,595.833333,147.608247,84.327404,157.762128,1.000000,1.000000,83.333333,104.986772,219.089023,72261.183922,0.479197,0.209864
2,1150,8.0,0.000000e+00,1.986629e+01,0.000000,161.194030,372.222222,197.593547,109.420241,391.780632,1.000000,1.000000,200.000000,274.543530,58.708705,50636.488945,4.676368,0.448129
3,1151,8.0,0.000000e+00,3.420076e+05,0.000000,163.636364,366.666667,44.095855,35.572912,45.133547,0.500000,0.166667,0.000000,29.658551,31.671540,2950.998393,0.936442,0.341413
4,1152,8.0,0.000000e+00,1.788171e+00,0.000000,66.176471,906.666667,35.901099,38.490018,58.794474,0.666667,0.444444,0.000000,41.573971,30.987408,4047.218364,1.341641,0.331016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,2709,2.0,1.591333e+05,8.385086e+04,1.897813,389.189189,154.166667,117.186343,116.633593,151.840560,0.916667,0.583333,33.333333,105.171083,101.131713,33414.393740,1.039942,0.405844
1774,2922,7.0,4.498071e+06,8.732365e+05,5.151034,501.030928,119.753086,69.837707,91.854958,98.657657,0.240000,0.120000,0.000000,69.602043,68.960536,15078.998305,1.009303,0.309502
3798,4946,8.0,1.314452e+05,1.702537e+06,0.077205,116.546763,514.814815,161.864988,137.68615,293.552107,0.888889,0.888889,83.333333,205.913935,142.339843,92079.318127,1.446636,0.323904
3852,5000,8.0,5.467302e+05,4.052008e+05,1.349282,107.462687,558.333333,112.731244,47.140452,205.480467,1.000000,1.000000,66.666667,33.333333,33.333333,3490.658504,1.000000,0.447928


In [ ]:
df2.to_csv(r"D:/CK/dataset.csv",mode="a",index=False,header=False)